In [1]:
import numpy as np
import matplotlib.pyplot as plt
from make_files import Mg_Shapes
import os
# from distutils.dir_util import copy_tree
import shutil

# Step 1: Initial Optical and Thermal Calculations

### Make Folders and Submit Scripts

In [2]:
%rm -r raster_data_H/*

In [3]:
boxsize = 80 # nm, rastering 
stepsize = 20
shapefile=str('shape.dat_little')
shape=np.loadtxt(str(shapefile), skiprows=7)

# Create ddscat.par files
mgshape = Mg_Shapes(lat_space=1, 
                shell=True)

yrange = np.arange(-boxsize/2, boxsize/2+1, stepsize)
zrange = np.arange(-boxsize/2, boxsize/2+1, stepsize)
ygrid, zgrid = np.meshgrid(yrange, zrange)
ypoints = np.ravel(ygrid); zpoints = np.ravel(zgrid)

In [4]:
for i in range(0, len(ypoints)):
    directory = str("y")+str(int(ypoints[i]))+str("_z")+str(int(zpoints[i]))
    os.mkdir(str("raster_data_H/")+str(directory))
    shutil.copyfile(shapefile, str("raster_data_H/")+str(directory)+str('/shape.dat'))
    mgshape.write_ddscatpar(shapefile=shapefile,
                            wavelength=0.785, # um
                            diel_paths=['/home/caw97/rds/hpc-work/diels/Mg_Palik.txt',
                                        '/home/caw97/rds/hpc-work/diels/MgO.txt'],
                            nrfld=2,
                            iwrksc=0,
                            nplanes=0,
                            gauss_center=np.array([ypoints[i], zpoints[i]])
                           )
    os.rename('ddscat.par', str("raster_data_H/")+str(directory)+str('/ddscat.par'))
    
    ## Prepare var.par file
    mgshape.write_varpar(wavelength=0.785, xvals=shape[:,1], yvals=shape[:,2], zvals=shape[:,3])
    os.rename('var.par', str("raster_data_H/")+str(directory)+str('/var.par'))

    
    
    

### Prepare Launch File

In [5]:
########################################################
##### Launch 1 #####

new_launch = open('launch_temp1.slurm')
lines = new_launch.readlines()

new_string_y = str('yarray=( ')+' '.join(repr(int(i)) for i in ypoints).replace("'", '"') + str(' )') + str('\n')
new_string_z = str('zarray=( ')+' '.join(repr(int(i)) for i in zpoints).replace("'", '"') + str(' )') + str('\n')

lines[25] = new_string_y
lines[27] = new_string_z

num=len(ypoints)
lines[21] = str('#SBATCH --array=0-')+str(num-1)
new_launch = open(str('launch1.slurm'),"w")
new_launch.writelines(lines)
new_launch.close()

########################################################
##### Launch 2 #####

shutil.copyfile('launch_temp2.slurm', 'launch2.slurm')


########################################################
##### Launch 3 #####

new_launch = open('launch_temp3.slurm')
lines = new_launch.readlines()

lines[25] = new_string_y
lines[27] = new_string_z

lines[21] = str('#SBATCH --array=0-')+str(num-1)
new_launch = open(str('launch3.slurm'),"w")
new_launch.writelines(lines)
new_launch.close()


########################################################
##### Launch 4 #####

new_launch = open('launch_temp4.slurm')
lines = new_launch.readlines()

lines[25] = new_string_y
lines[27] = new_string_z

lines[21] = str('#SBATCH --array=0-')+str(num-1)
new_launch = open(str('launch4.slurm'),"w")
new_launch.writelines(lines)
new_launch.close()


########################################################
##### Launch 5 #####

shutil.copyfile('launch_temp5.slurm', 'launch5.slurm')






'launch5.slurm'